In [ ]:
### Laddar in modeller som ska kombineras
from keras.models import load_model

face = load_model(r'C:\Users\leona\Desktop\Data Science\face.tf')
speech = load_model(r'C:\Users\leona\Desktop\Data Science\speechgg2lt.tf')

In [ ]:
# För att undvika namnkonflikter när modellerna kombineras, lägg till ett suffix på alla lager i talmodellen.
# Detta är för att hantera en potentiell bugg i Keras.
for i, layer in enumerate(speech.layers):
    layer._name = layer._name + str("_1")

In [ ]:
# Checkar att namnen har ändrats
for i, layer in enumerate(speech.layers):
    print(layer, layer.name)

In [ ]:
# Laddar in bilddata som användes från modellen innan. 

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os

# Importing Deep Learning Libraries

from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

picture_size = 48
folder_path = 'D:/DataSets/images/images/Images Merge'

batch_size = 128

datagen_train = ImageDataGenerator()
datagen_val = ImageDataGenerator()

### https://stackoverflow.com/questions/42284873/assign-imagedatagenerator-result-to-numpy-array
### https://www.bing.com/search?q=keras+ImageDataGenerator+to+np.array&qs=n&form=QBRE&sp=-1&lq=0&pq=keras+imagedatagenerator+to+np.array&sc=11-36&sk=&cvid=64B6BA2699D84DCDA7653B0DE6C00DE6&ghsh=0&ghacc=0&ghpl=

train_set = datagen_train.flow_from_directory(folder_path+'train',
                                              target_size = (picture_size, picture_size),
                                              color_mode ='grayscale',
                                              batch_size = batch_size,
                                              class_mode = 'categorical',
                                              shuffle = True)

test_set = datagen_train.flow_from_directory(folder_path+'validation',
                                              target_size = (picture_size, picture_size),
                                              color_mode = 'grayscale',
                                              batch_size = batch_size,
                                              class_mode = 'categorical',
                                              shuffle = False)

In [ ]:
# Konvertera data till numpy arrays för att säkerställa att all data som matas in i modellen har samma datatype
# Datan kommer att vara numpy arrays.

x_train=np.concatenate([train_set.next()[0] for i in range(train_set.__len__())])
y_train=np.concatenate([train_set.next()[1] for i in range(train_set.__len__())])
print(x_train.shape)
print(y_train.shape)

In [ ]:
# Konvertera data till numpy arrays för att säkerställa att all data som matas in i modellen har samma datatype
# Datan kommer att vara numpy arrays.

x_test=np.concatenate([test_set.next()[0] for i in range(test_set.__len__())])
y_test=np.concatenate([test_set.next()[1] for i in range(test_set.__len__())])
print(x_test.shape)
print(y_test.shape)

In [ ]:
train_set

In [ ]:
#type(train_set)
type(test_set)

In [ ]:
paths = []
labels = []

def sorting_sounds(emotion):
    # Laddar datasetet och extraherar filvägar och etiketter från filnamnen
#labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    x = 0
    # Traversera alla filer i datasetets katalogstruktur
    for dirname, _, filenames in os.walk(r'D:\DataSets\AudioWAV'):
        for filename in filenames:
            # Lagra fullständig filväg

            label = filename.split('_')[-2]
            if label == emotion and not label == 'DIS':
                paths.append(os.path.join(dirname, filename))
                labels.append(label.lower())

            elif label == 'DIS' and emotion == 'DIS' and x < 436:
                paths.append(os.path.join(dirname, filename))
                labels.append(label.lower())
                x += 1
            # Extrahera etikett från filnamnet
            

                
            #label = label.split('.')[0]
            #print(label)
            
        print('Dataset is loaded')

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio

sorting_sounds('ANG')
sorting_sounds('DIS')
sorting_sounds('FEA')
sorting_sounds('HAP')
sorting_sounds('NEU')
sorting_sounds('SAD')

## create a dataframe
df = pd.DataFrame()
df['speech'] = paths
df['labels'] = labels

df['labels']= df['labels'].replace({'ps': 'surprise'})

def extract_mfcc(filename):
  y, sr = librosa.load(filename, duration=3, offset=0.5)
  mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
  return mfcc

X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))

X = [x for x in X_mfcc]
X = np.array(X)
X = np.expand_dims(X, -1)

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['labels']])
y = y.toarray()


In [ ]:
# Laddar in ljuddata som ska användas för att träna den nya modellen

#Audio data
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio

paths = []
labels = []
for dirname, _, filenames in os.walk('D:\DataSets\TESS Toronto emotional speech set data 2'):
  for filename in filenames:
    paths.append(os.path.join(dirname, filename))
    label = filename.split('_')[-1]
    label = label.split('.')[0]
    labels.append(label.lower())
print('Dataset is loaded')

## create a dataframe
df = pd.DataFrame()
df['speech'] = paths
df['labels'] = labels

df['labels']= df['labels'].replace({'ps': 'surprise'})

def extract_mfcc(filename):
  y, sr = librosa.load(filename, duration=3, offset=0.5)
  mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
  return mfcc

X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))

X = [x for x in X_mfcc]
X = np.array(X)
X = np.expand_dims(X, -1)

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['labels']])
y = y.toarray()



In [ ]:
X[0].shape

In [ ]:
# Detta görs för att kombinera de två sparade modellerna för ansikts- och taligenkänning

from keras.layers import Concatenate, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras import metrics

# Skapar en ny modell som kombinerar utgångarna från ansikts- och talmodellerna
merged_output = Concatenate()([face.output, speech.output])

# Första nya fullt anslutna lagret för att kombinera funktioner från båda modellerna
new_layer = Dense(256, activation='relu')(merged_output)
new_layer = Dropout(0.2)(new_layer)  # Förhindrar överanpassning

# Andra nya fullt anslutna lagret för ytterligare funktionkombination
new_layer = Dense(128, activation='relu')(new_layer)

# Utgångslager med softmax-aktivering för klassificering i 6 kategorier
new_layer = Dense(6, activation='softmax')(new_layer)

# Skapar den nya kombinerade modellen med båda ingångarna och det nya utgångslagret
new_model = Model([face.input, speech.input], new_layer)

# Kompilering av den nya modellen med Adam-optimizer och kategorisk korsentropi som förlustfunktion
opt = Adam(learning_rate=0.0001)
new_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy])

# Sammanfattar arkitekturen för den nya kombinerade modellen
new_model.summary()



In [ ]:
# Sparar modellen så att den kan användas senare, samt tillämpar olika regulariseringsmetoder för att förbättra modellen

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Checkpoint för att spara den bästa modellen baserat på valideringsnoggrannhet
checkpoint = ModelCheckpoint("./fn.tf", monitor="val_categorical_accuracy", verbose=1, save_best_only=True, mode='max')

# Early stopping för att avbryta träningen när noggrannheten slutar förbättras för att undvika överträning
early_stopping = EarlyStopping(monitor='categorical_accuracy',
                               min_delta=0,
                               patience=3,
                               verbose=1,
                               restore_best_weights=True)

# Reduce learning rate on plateau för att minska inlärningshastigheten när noggrannheten slutar förbättras
reduce_learningrate = ReduceLROnPlateau(monitor='categorical_accuracy',
                                        factor=0.2,
                                        patience=3,
                                        verbose=1,
                                        min_delta=0.0001)

callbacks_list = [early_stopping, checkpoint, reduce_learningrate]

epochs = 48

In [ ]:
# Tränar modellen med träningsdata och validerar med valideringsdata
# Använder callbacks för att spara bästa modell, avbryta vid överträning och justera inlärningshastigheten

history = new_model.fit(          
    x=[x_train, X],
    y=[y_train, y], 
    steps_per_epoch= 35, ###train_set.n//train_set.batch_size,
    epochs=epochs, 
    validation_split=0.20,
    validation_steps= 9, ###test_set.n//test_set.batch_size,
    callbacks=callbacks_list
    )

In [ ]:
## Visualiserar tränings- och valideringsförlust samt noggrannhet för att bedöma modellens prestanda och avgöra om ytterligare träning behövs

plt.style.use('dark_background')

plt.figure(figsize=(20, 10))

# Plot för tränings- och valideringsförlust

plt.subplot(1, 2, 1)
plt.title('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

# Plot för tränings- och valideringsnoggrannhet
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['categorical_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_categorical_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
### Säkerställer att modellen fungerar med rätt dimensioner
xf = x_test[150]
xs = X[150]

xf = np.expand_dims(xf, axis=0)
xs = np.expand_dims(xs, axis=0)

new_model.predict([xf, xs])[0]